In [266]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt
from top2vec import Top2Vec
from nltk.corpus import stopwords
from gensim.models import TfidfModel
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import load_iris
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import json

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [267]:
training_data = pd.read_csv("training_data/training_data.csv")
test_data = pd.read_csv("test_data/test_data.csv")

In [268]:
training_data.target.value_counts()

0    925
1    155
Name: target, dtype: int64

In [269]:
test_data.target.value_counts()

0    395
1     68
Name: target, dtype: int64

In [270]:
################################################### bert ##############################################################################

In [271]:
bert_train_vecs = pd.read_csv("vecs/bert_train_vecs.csv")
bert_train_vecs = bert_train_vecs.values.tolist()

In [272]:
bert_test_docs = test_data.allegation_desc
bert_test_docs = json.loads(bert_test_docs.to_json(orient='records'))

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
bert_test_vecs = sentence_model.encode(bert_test_docs, show_progress_bar=True)

Batches: 100%|██████████| 15/15 [00:13<00:00,  1.14it/s]


In [273]:
################################################### gensim ##############################################################################

In [274]:
gensim_train_vecs = pd.read_csv("vecs/gensim_train_vecs.csv")
gensim_train_vecs = gensim_train_vecs.values.tolist()

In [275]:
gensim_model = gensim.models.ldamodel.LdaModel.load("models/gensim_train.model")

In [276]:
gensim_test_docs = test_data.allegation_desc

def lemmatization(descs, allowed_pos_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    final_text = []
    for desc in descs:
        doc = nlp(desc)
        new_text = " ".join([token.lemma_ for token in doc if token.pos_ in allowed_pos_tags])
        final_text.append(new_text)
    return (final_text)

lemmatized_texts = lemmatization(gensim_test_docs)

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return list(bigram[doc] for doc in texts)

def make_trigrams(texts):
    return list(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_words)

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

test_corpus = [id2word.doc2bow(text) for text in texts]

tdidf = TfidfModel(test_corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tdif = []

for i in range(0, len(test_corpus)):
    bow = test_corpus[i]
    low_value_words = []
    tdif_ids = [id for id, value in tdidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tdidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tdif
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tdif = [id for id in bow_ids if id not in tdif_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tdif]
    test_corpus[i] = new_bow

def get_test_vecs():
    gensim_test_vecs = []
    for i in range(len(gensim_test_docs)):
        top_topics = gensim_model.get_document_topics(test_corpus[i], minimum_probability=0.0)
        topic_vec = [top_topics[i][1] for i in range(10)]
        topic_vec.extend([len(gensim_test_docs.iloc[i])]) # length review
        gensim_test_vecs.append(topic_vec)
    return gensim_test_vecs

In [277]:
gensim_test_vecs = get_test_vecs()

In [278]:
################################################### top2vec ##############################################################################

In [279]:
t2v_train_model = Top2Vec.load("models/top2vec_train_model")
t2v_train_vecs = t2v_train_model.document_vectors

In [280]:
def convert_test_data_to_list(data):
    t2v_test_docs = [x for x in data["allegation_desc"]]
    return t2v_test_docs

In [281]:
t2v_test_docs = convert_test_data_to_list(test_data)
t2v_test_model = Top2Vec(t2v_test_docs, embedding_model_path="models/top2vec_train_model")
t2v_test_vecs = t2v_test_model.document_vectors

2023-02-01 14:14:37,093 - top2vec - INFO - Pre-processing documents for training
2023-02-01 14:14:37,150 - top2vec - INFO - Creating joint document/word embedding
2023-02-01 14:14:38,097 - top2vec - INFO - Creating lower dimension embedding of documents
2023-02-01 14:14:43,013 - top2vec - INFO - Finding dense areas of documents
2023-02-01 14:14:43,060 - top2vec - INFO - Finding topics


In [282]:
############################################################# regression #################################################################

In [283]:
# X_test = np.array(t2v_test_vecs)
# y_test = np.array(test_data.target)
# X = np.array(t2v_train_vecs)
# y = np.array(training_data.target)
# y = np.pad(y, pad_width=((463,0)), mode='constant')

In [284]:
# X_test = np.array(gensim_test_vecs)
# y_test = np.array(test_data.target)
# X_test = np.pad(X_test, pad_width=((0,0),(0,10)), mode='constant')
# X = np.array(gensim_train_vecs)

# y = np.array(training_data.target)
# y = np.pad(y, pad_width=((463,0)), mode='constant')

In [285]:
X_test = np.array(bert_test_vecs)
y_test = np.array(test_data.target)
X = np.array(bert_train_vecs)

y = np.array(training_data.target)
y = np.pad(y, pad_width=((463,0)), mode='constant')

In [286]:
kf = KFold(5, shuffle=True, random_state=48)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True,
    ).fit(X_train_scale, y_train)

    test_data["lr_scores"] = lr.predict(scaler.transform(X_test))
    test_data["lr_scores_prob_1"] = lr.predict_proba(scaler.transform(X_test))[:, 1]

    y_pred = lr.predict(scaler.transform(X_val_scale))
    cv_lr_f1.append(f1_score(y_val, y_pred, average='weighted'))
    
    # Logistic Regression Mini-Batch SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)

    test_data["sgd_scores"] = sgd.predict(scaler.transform(X_test))
    test_data["sgd_scores_prob_1"] = sgd.predict_proba(scaler.transform(X_test))[:, 1]

    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average="weighted"))
    
    # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)

    test_data["sgd_huber_scores"] = sgd_huber.predict(scaler.transform(X_test))
    test_data["sgd_huber_scores_prob_1"] = sgd_huber.predict_proba(scaler.transform(X_test))[:, 1]
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average="weighted"))

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')

Logistic Regression Val f1: 0.564 +- 0.019
Logisitic Regression SGD Val f1: 0.776 +- 0.026
SVM Huber Val f1: 0.831 +- 0.028


In [287]:
test_pred = test_data[test_data.target.astype(str).isin(["1"])]
test_pred.lr_scores.value_counts()

0    59
1     9
Name: lr_scores, dtype: int64